## Foursquare Location Matching
https://www.kaggle.com/competitions/foursquare-location-matching

#### About notebook
This is my first kaggle experience. 
I've been not proud of my results (top 40% on date of writing) but then I found out that this competition had data leakage so now I'm just confused :)
I'm very grateful to kaggle community for utility functions implemented in cython and discovery of unidecode.
Also I've used this companies dataset https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset

#### Challenge description
We are given dataset representing geographical locations: restaurants, caffes, shops etc. Same location (identified by Point Of Interest (POI)) can be represented by multiple (from 1 to 100+) entries (identified by unique ID).
Our target is to match all ID's by same POI in format:
<table>
    <th><tr><td><b>id</b></td>
        <td><b>matches</b></td></tr></th>
    <tr><td>E_000001272c6c5d</td><td>E_000001272c6c5d E_da7fa3963561f8</td></tr>
    <tr><td>E_000008a8ba4f48</td><td>E_000008a8ba4f48</td></tr>
</table>
Quality of our prediction is measured by Mean IoU score. 
Train dataset contains 1100k+ rows. Test dataset contains 600k rows

#### Exploratiove data analysis (summary only)
1. We are provided with location name, address, latitude, longtitude, city, state, zip, country, url, phone, categories.
2. Name, coordinates and country always present, others often omitted:
    > ##### non-nan values share
    > categories           0.9137<br>
    > city                 0.7373<br>
    > address              0.6517<br>
    > state                0.6307<br>
    > zip                  0.4772<br>
    > phone                0.3011<br>
    > url                  0.2351<br>
3. Most of POI has no more than 1 match (not taking itself into account). It means that it can be useful to manually limit number of predicted matches to decrease average false positive rate. <br> 
<table>
    <th><tr><td><b>Number of matches</b></td>
            <td><b>Share %</b></td>
            <td><b>Cumulative share %</b></td></tr></th>
    <tr><td>1</td><td>37.32</td><td>37.32</td></tr>
    <tr><td>2</td><td>48.77</td><td>86.10</td></tr>
    <tr><td>3</td><td>6.38</td><td>92.47</td></tr>
    <tr><td>4</td><td>2.09</td><td>94.56</td></tr>
    <tr><td>5</td><td>1.11</td><td>95.66</td></tr>
</table>

4. Many names and addresses contain numbers, therefore text preprocessing stage must either extract them into separate feature or convert to words (i.e. 1 - > "one", 2 -> "two").
5. Names and addresses contain multilanguage unicode characters so there must be found method to unify them.
6. Sometimes match can be found by comparison not only name-to-name but name-to-addres, for example when some caffe is situated in shopping mall and has POI of that mall. In some cases address of this caffe will be equal to mall's name.
7. Most of matches located nearby, nearest neighbors matching with 50 neighbors can cover more than 90% of them.
8. POI's countries never intersect, so it's good to group dataset by country before performing matching.

#### My approach description

##### Training
1. Unify text using unidecode, leave only alphabetical characters.
2. I've used external dataset of companies to extract company names from "name" feature. 
3. Leave only numbers for phones.
4. Take sample of 600k rows (sampling stratified by POI) to model distribution of false positives in test set. Leave others for testing.
5. Groupbed by countries, find 50 neighbors by location data and 30 neighbors by name similarity. For name similarity I've used ANN with angular distance over 256-dim word-boundary-3-gram bag-of-words with hash trick.
6. Make features, including: angular distance, string similarities, category iou, company_names iou, ranking features, numbers iou.
7. Fit LightGBM using GP for hyperparameters optimization on that pairs.
8. Grid-searching best threshold amoung 3 approaches: a) one score threshold b) score threshold and max amount of neighbors c) separate threshold for each score rank (e.g 0.4 for 1st, 0.5 for 2nd, 0.6 for 3rd). Best found approach is b), threshold 0.5 and no more than 3 matches including itself.

##### Prediction
1. Perform same preprocessing and neighbors matching as during train, predict scores and leave final predictions.

#### My experience and insights after trying different approaches
1. Unidecode works well for romance languages but much worse for languages such as arabian, chinese and so on. So my approach almost never can match company source name and it's proper latin transcription. I think this can be solved either by using different transcriprion libraries for different languages or atleast using external vocabulary for companies with their names in source language.
2. Multilanguage models have not worked well for me (I've tried small multilanguage models limited by laptop capabilities). Theis usage is good for categories to find similar ones, but for name matching exact character comparison is more important. From other hand, LM features useally not correlated to BOW similarity, so their usage would definitely increase overall score.
3. Phonetics dmetaphone also provides good uncorrelated features which usage can increase final metric more than 1% in my case.
4. Angular metric for embedding comparison is much more precise than euclidean or cosine, that's why I had to use hash trick to convert sparce matrics to dense arrays to use ANN on them.
5. Matching by name similarity together with location increases matching recall from 91% to almost 98%.
6. Ranking features give big perfomance boost, by ranking features I mean: if we have neighbors with name similarity 0.1, 0.2, 0.05, the features value will be 1, 0, 2.
7. The most important feature for me became BOW similarity, I could get better score by increasing dimensionality from 256 to 1024 but there is tradeoff of execution time.

In [ ]:
!pip install /kaggle/input/foursqare-matching-comp-libs/textdistance-4.2.2-py3-none-any.whl --user

In [ ]:
%load_ext cython

In [ ]:
TEST_PATH = '/kaggle/input/foursquare-location-matching/test.csv'
COMPANIES_PATH = '/kaggle/input/free-7-million-company-dataset/companies_sorted.csv'
MODEL_PATH = '/kaggle/input/foursqare-matching-comp-libs/model.jl'
OUTPUT_PATH = 'submission.csv'

import os, psutil
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def cpu_stats():
    """ Shows memory usage """
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))


# seed everything
import os 
import random
import numpy as np 

seed = 2022

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import gc
import joblib
from unidecode import unidecode
import lightgbm as lgb
import scipy.sparse as sp
import re
from collections import Counter
from time import time
import numba as nb
from difflib import SequenceMatcher
import warnings
from functools import wraps
from time import time
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from itertools import combinations

import textdistance
from pynndescent import NNDescent

In [19]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        # print('func:%r args:[%r, %r] took: %2.4f sec' % (f.__name__, '_', kw, te-ts))
        return result
    return wrap


def fmt_matches(df, pairs, mask=None):
    """ formats pairs for submission """
    p = pairs.copy()
    if mask is not None: p.loc[~mask, 'id_2'] = ''
    matches = p[p.id_1 != p.id_2].sort_values(['id_1', 'id_2']).groupby('id_1', as_index=False).id_2.apply(lambda x: ' '.join(x)) 
    matches.columns = ['id', 'matches']
    matches.matches = matches.matches.str.replace(r' {2,}', '').str.strip()
    matches = pd.concat([matches, pd.DataFrame({'id':df.id[~df.id.isin(matches.id).values]})], axis=0)
    matches.reset_index(inplace=True, drop=True)
    matches.matches.fillna('', inplace=True)
    matches['matches'] = (matches['id'] + ' ' + matches['matches']).str.strip()
    matches = matches.set_index('id').loc[df.id].reset_index()
    return matches

def AvgIoUScore(matches_pred, matches_true):
    """ target metric """
    scores = []
    for m1,m2 in zip(matches_pred, matches_true):
        m1_ = set(m1.split(' '))
        m2_ = set(m2.split(' '))
        if len((m1_ | m2_)) == 0: continue
        i = len(m1_ & m2_)
        u = len(m1_ | m2_)
        score = i/u
        scores.append(score)
    return np.array(scores)

def compute_score(data, pairs):
    """ applies metric to pairs """
    x = pd.merge(data, data, suffixes=('_1', '_2'), on='point_of_interest')
    T = fmt_matches(x, (x.id_1 !=x.id_2)).sort_values('id')
    P = fmt_matches(pairs, pairs.match & (pairs.id_1 != pairs.id_2)).sort_values('id')
    print(T.shape, P.shape)
    T = T[T.id.isin(P.id)]
    t = T.matches.values
    p = P.matches.values
    print(t.shape, p.shape)
    scores = AvgIoUScore(t, p)
    return T, P, scores.mean()

def get_total_matches(df):
    counts = df.groupby('point_of_interest').count()
    matches = counts * (counts - 1) // 2
    return matches.sum()['id']

def get_percent_matches(df, pairs):
    """ I used it to select best number of neighbors """
    matches_tot = get_total_matches(df)
    n_matches = len(pairs.loc[pairs.match]) / 2
    print(f'{100 * n_matches / matches_tot}% of {matches_tot} matches found')
    return n_matches / matches_tot

def preprocess_text(s:pd.Series, keep_commas=False, keep_numbers=False):
    # normalize unicode
    s_new = s.fillna('').apply(lambda x: unidecode(x)).str.lower()

    s_new = s_new.str.replace('&', ' and ')
    s_new = s_new.str.replace(r'[^a-z 0-9,]' if keep_commas else r'[^a-z 0-9]', '', regex=True)

    # separate numbers
    if keep_numbers:
        s_new = s_new.str.replace(r'(\b[a-z]+)([0-9]+\b)', r'\1 \2', regex=True).str.replace(r'(\b[0-9]+)([a-z]+\b)', r'\1 \2', regex=True)
    else: s_new = s_new.str.replace('[0-9]', '', regex=True)
    
    s_new = s_new.str.replace(' +', ' ', regex=True) 
    if keep_commas: s_new = s_new.str.replace(' ?, ?', ', ', regex=True)
    return s_new.str.strip()

In [ ]:
%%cython
import numpy as np  # noqa

cpdef int FastLCS(str S, str T):
    cdef int i, j
    cdef int cost
    cdef int v1,v2,v3,v4
    cdef int[:, :] dp = np.zeros((len(S) + 1, len(T) + 1), dtype=np.int32)
    for i in range(len(S)):
        for j in range(len(T)):
            cost = (int)(S[i] == T[j])
            v1 = dp[i, j] + cost
            v2 = dp[i + 1, j]
            v3 = dp[i, j + 1]
            v4 = dp[i + 1, j + 1]
            dp[i + 1, j + 1] = max((v1,v2,v3,v4))
    return dp[len(S)][len(T)]

In [ ]:
@nb.njit('float64[:](float32[:,:], float32[:,:])')
def angular(a, b):
    assert a.shape == b.shape
    n, m = a.shape
    res = np.empty(n, dtype=np.float64)
    for i in range(n):
        s = 0.0
        for j in range(m): s += a[i,j] * b[i,j]
        if s > 1: s = 1
        res[i] = np.arccos(s) / np.pi * 2
    return res

def angular_impute(a,b,empty_mask,impute=2):
    d = angular(a,b)
    d[empty_mask] = -1
    return d

@timing
def lcs(A, B, impute_value=-1):
    lcs = []
    for a,b in zip(A,B):
        l = min(len(a), len(b))
        if l == 0: lcs.append(impute_value); continue
        lcs.append(FastLCS(a,b)/len(a))
    return np.array(lcs)

@timing
def lccs(A,B, impute_value=-1):
    # true commong contiguous substring
    lccs = []
    for a,b in zip(A,B):
        l = min(len(a), len(b))
        if l == 0: lccs.append(impute_value); continue
        lccs.append(SequenceMatcher(None, a, b).quick_ratio())
    return np.array(lccs)

@timing
@nb.njit('float64[:](float32[:], float32[:],float32[:],float32[:])')
def haversine(phi_1, lambda_1, phi_2, lambda_2):
    a = np.sin((phi_2 - phi_1) / 2.0) ** 2 + np.cos(phi_1) * np.cos(phi_2) * np.sin((lambda_2 - lambda_1) / 2.0) ** 2
    return  637.1 * 2 * np.arcsin(np.sqrt(a)) # output distance in kilometers

@timing
def levenshtein(A, B, impute_value=2):
    d = []
    for a,b in zip(A, B):
        l = max(len(a), len(b))
        if l == 0: d.append(impute_value); continue
        d.append(textdistance.levenshtein(a,b)/l)
    return np.array(d)

@timing
def iou(A,B, impute_value=-1):
    d = []
    for a,b in zip(A,B):
        u = len(a | b)
        if u == 0: d.append(impute_value); continue
        i = len(a & b)
        d.append(i/u)
    return np.array(d)

@nb.njit('int32[:](int32[:])')
def ranker_inner(index):
    idx0 = -1
    c = 0
    R = np.zeros(index.shape[0], dtype=np.int32)
    for i,idx in enumerate(index):
        if(idx == idx0):
            c += 1
        else:
            idx0 = idx
            c = 0
        R[i] = c
    return R

@timing
def ranker(index):
    """ converts features to ranks """
    return ranker_inner(LabelEncoder().fit_transform(index).astype(np.int32))

In [ ]:
def nn_iter(df, location_data:np.array, chunk_size=25000, n_neighbors=6, train=False, ann=False):
    """
        finds neighbors
        if dataset is small it is faster to generate all combinations
        in other case we use NN and ANN.
    """
    if (df.shape[0] <= 50):
        comb = list(combinations(df.chunk_id.values, 2))
        comb = np.stack(comb).astype(np.uint32)
        df_1 = df.iloc[comb[:, 0]].reset_index(drop=True)
        df_1.columns += '_1'
        df_2 = df.iloc[comb[:, 1]].reset_index(drop=True)
        df_2.columns += '_2'
        pairs = pd.concat([df_1, df_2], axis=1)
        if train:
            pairs['match'] = pairs['point_of_interest_1'].values == pairs['point_of_interest_2'].values
        else: pairs['match'] = np.nan
        yield pairs
    else:
        if ann:
            nn = NNDescent(location_data, n_neighbors=30, metric='true_angular',pruning_degree_multiplier=1.5)
            nn.prepare()
        else:
            nn = NearestNeighbors(n_neighbors=min(n_neighbors, len(df)), 
                                   algorithm='kd_tree', n_jobs=-1)
            nn.fit(location_data)

        cols_1 = dict(zip(df.columns, [f'{col}_1' for col in df.columns]))
        cols_2 = dict(zip(df.columns, [f'{col}_2' for col in df.columns]))

        for i in range(0, df.shape[0], chunk_size):
            if ann:
                neighbors_array, _ = nn.query(location_data[i:i+chunk_size], n_neighbors, epsilon=0.2)
            else:
                neighbors_array = nn.kneighbors(location_data[i:i+chunk_size], return_distance=False)

            df_1 = df.iloc[i:i+chunk_size].copy()
            df_1.reset_index(inplace=True, drop=True)
            df_1['orig_index'] = np.arange(df_1.shape[0])
            df_1 = pd.concat(n_neighbors * [df_1], ignore_index=True)
            df_1.reset_index(inplace=True)
            df_1.rename(columns=cols_1, inplace=True)
            df_1.sort_values(['orig_index', 'index'], inplace=True)
            df_1.drop(columns=['orig_index', 'index'], inplace=True)
            df_1.reset_index(drop=True, inplace=True)

            idxs = neighbors_array.flatten()
            df_2 = df.iloc[idxs]
            df_2.rename(columns=cols_2, inplace=True)
            df_2.reset_index(inplace=True, drop=True)

            pairs = pd.concat([df_1, df_2], axis=1)

            del df_1, df_2
            gc.collect()
            pairs = pairs[pairs.id_1.values != pairs.id_2.values]
            pairs.reset_index(drop=True, inplace=True)
            if train:
                pairs['match'] = pairs['point_of_interest_1'].values == pairs['point_of_interest_2'].values
            else: pairs['match'] = np.nan
            yield pairs

In [ ]:
warnings.filterwarnings('ignore')

@nb.jit(nopython=False)
def hash_tokenize(strings, n_features, expr=r"(?=([a-z][a-z][a-z]))", normalize=False, dtype=np.float32):
    # implements hash trick, faster than sklearn.HashVectorizer
    # I do it manually because it's fast and surely deterministic
    expr = re.compile(expr)
    cols, rows, data = [], [], []
    i = 0
    for s in strings:
        matches = re.findall(expr, s)
        for t in matches:
            cols.append(hash(t) % n_features); rows.append(i); data.append(1)
        if len(matches) == 0:
            cols.append(0); rows.append(i); data.append(0)
        i+=1
    matrix_shape = (len(strings), n_features)
    arr = np.zeros(matrix_shape, dtype=dtype)
    sp.coo_matrix((np.array(data, dtype=dtype),(rows, cols)), dtype=dtype, shape=matrix_shape).toarray(out=arr)
    # normalize
    if normalize:
        arr = arr / (np.linalg.norm(arr, axis=1)[:, None] + 1e-12)
    return arr

In [ ]:
def make_features_inplace(X, e_name):
    # distance
    X.loc[:, 'distance'] = haversine(X.phi_1.values, X.lambda_1.values, X.phi_2.values, X.lambda_2.values)
    
    # ngram distance
    X.loc[:, 'name_ngram_dist'] = angular_impute(e_name[X.chunk_id_1.values], e_name[X.chunk_id_2.values], ((X.name_1.str.len() == 0) | (X.name_2.str.len() == 0)).values, 2)
    # iou 
    X.loc[:, 'name_numbers_iou'] = iou(X.name_numbers_1.values, X.name_numbers_2.values)
    X.loc[:, 'addr_numbers_iou'] = iou(X.addr_numbers_1.values, X.addr_numbers_2.values)
    X.loc[:, 'cats_iou'] = iou(X.cats_1.values, X.cats_2.values)
    X.loc[:, 'probable_company'] = iou(X.probable_company_1.values, X.probable_company_2.values)
    
    #lccs
    X.loc[:, 'name_lccs'] = lccs(X['name_1'], X['name_2'])
    X.loc[:, 'addr_lccs'] = lccs(X['addr_1'], X['addr_2'])
    X.loc[:, 'state_lccs'] = lccs(X['state_1'], X['state_2'])
    X.loc[:, 'city_lccs'] = lccs(X['city_1'], X['city_2'])
    X.loc[:, 'zip_lccs'] = lccs(X['zip_1'], X['zip_2'])
    X.loc[:, 'url_lccs'] = lccs(X['url_1'], X['url_2'])
    X.loc[:, 'phone_lccs'] = lccs(X['phone_1'], X['phone_2'])
    
    # lcs
    X.loc[:, 'name_lcs'] = lcs(X['name_1'], X['name_2'])
    X.loc[:, 'addr_lcs'] = lcs(X['addr_1'], X['addr_2'])
    X.loc[:, 'city_lcs'] = lcs(X['city_1'], X['city_2'])
    
    
    # levenshtein
    X.loc[:, 'name_levenshtein'] = levenshtein(X['name_1'], X['name_2'])
    X.loc[:, 'addr_levenshtein'] = levenshtein(X['addr_1'], X['addr_2'])
    X.loc[:, 'city_levenshtein'] = levenshtein(X['city_1'], X['city_2'])
    
    X.sort_values(['id_1', 'distance'], inplace=True)
    X.loc[:, 'distance_rnk'] = ranker(X.id_1.values)
    X.sort_values(['id_1', 'name_lccs'], inplace=True, ascending=False)
    X.loc[:, 'name_lccs_rnk'] = ranker(X.id_1.values)
    X.sort_values(['id_1', 'name_ngram_dist'], inplace=True, ascending=False)
    X.loc[:, 'name_ngram_dist_rnk'] = ranker(X.id_1.values)
    X.sort_values(['id_1', 'name_levenshtein'], inplace=True, ascending=False)
    X.loc[:, 'name_levenshtein_rnk'] = ranker(X.id_1.values)

In [ ]:
# %%time
data = pd.read_csv(TEST_PATH)
for c in ['name', 'address', 'city', 'state', 'zip', 'country', 'url', 'phone', 'categories']:
    try: data[c] = data[c].round()
    except: ...
    data[c] = data[c].fillna('').astype(str)

data['table_id'] = np.arange(len(data), dtype=np.int32)

data['phi'] = np.radians(data.latitude, dtype=np.float32)
data['lambda'] = np.radians(data.longitude, dtype=np.float32)

data['name_src'] = data.name
data['addr_src'] = data.address

data['name_numbers'] = data.name.str.findall(r'[0-9]+').apply(set)
data['addr_numbers'] = data.address.str.findall(r'[0-9]+').apply(set)

# need to be done on train in prod

data['name'] = preprocess_text(data.name_src)
data['addr'] = preprocess_text(data.addr_src)
data['city'] = preprocess_text(data.city)
data['state'] = preprocess_text(data.state)

data['phone'] = data.phone.str.replace('[^0-9]', '', regex=True)

# split categories
data['cats'] = preprocess_text(data.categories, keep_commas=True).str.split(', ').apply(set)

#probable company
companies = pd.read_csv(COMPANIES_PATH)

names = data.name_src.str.lower().str.split(' ').explode()
names = names[names.isin(companies.name.values)]
data['probable_company'] = names.groupby(by=names.index).apply(set)
data.loc[data.probable_company.isna(), 'probable_company'] = data.loc[data.probable_company.isna(), 'probable_company'].apply(lambda x: set()).values

del companies
gc.collect()

data['point_of_interest'] = 0

In [ ]:
gbm = joblib.load(MODEL_PATH)

x_cols = ['distance',
       'name_ngram_dist','name_numbers_iou',
       'addr_numbers_iou', 'cats_iou', 'probable_company', 'name_lccs',
       'addr_lccs', 'state_lccs', 'city_lccs', 'zip_lccs', 'url_lccs',
       'phone_lccs', 'name_lcs', 'addr_lcs', 'city_lcs', 'name_levenshtein',
       'addr_levenshtein', 'city_levenshtein', 'distance_rnk', 'name_lccs_rnk',
       'name_ngram_dist_rnk', 'name_levenshtein_rnk']

In [ ]:
warnings.filterwarnings('ignore')

pairs = []

data.reset_index(drop=True, inplace=True)
with tqdm(total=data.shape[0]) as pbar:
    for country, df in data.groupby('country'):
        if df.shape[0] == 1: 
            pbar.update(1) 
            continue
        chunk_size = min(df.shape[0], 10000)
        n_neighbors_loc = min(50, df.shape[0])
        n_neighbors_name = min(30, df.shape[0])
        
        ann = df.shape[0] > 1000
        
        df['chunk_id'] = np.arange(df.shape[0])
        
        location_data = df[['latitude', 'longitude']].values
        name_embeddings = hash_tokenize(df.name, 256, expr=r"(?=([a-z]{3,3}))", normalize=True, dtype=np.float32)
        
        print(country, df.shape[0], cpu_stats())
        
        for pairs_loc, pairs_name in zip(nn_iter(df, location_data, chunk_size, n_neighbors_loc, True, False), \
                                         nn_iter(df, name_embeddings, chunk_size, n_neighbors_name, True, ann)):
            pairs_ = pd.concat([pairs_loc, pairs_name], axis=0)
            pairs_.drop_duplicates(['id_1', 'id_2'], inplace=True)
            if pairs_.shape[0] == 0:
                pbar.update(df.shape[0]) 
                continue
            pairs_['country'] = country
            make_features_inplace(pairs_, name_embeddings)
            pairs_['score'] = gbm.predict(pairs_[x_cols])
            pairs_ = pairs_[['id_1', 'id_2', 'score', 'match']]
            pairs_.sort_values(['id_1', 'score'], inplace=True, ascending=False)
            pairs_['rnk'] = pairs_.groupby('id_1').cumcount().values
            pairs_ = pairs_[(pairs_.score >= 0.5) & (pairs_.rnk <= 2)]
            if pairs_.shape[0] == 0: 
                pbar.update(df.shape[0]) 
                continue
            pairs.append(pairs_)
              
            
            del pairs_
            gc.collect()
        pbar.update(df.shape[0])
        del location_data, name_embeddings
        gc.collect()

In [ ]:
%%time
if len(pairs) > 0:
    submission = fmt_matches(data, pd.concat(pairs, axis=0))
else:
    submission = data[['id']]
    submission['match'] = submission.id.values
submission.to_csv(OUTPUT_PATH, index=False)